importとか諸々

In [74]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

データの読み込み

In [79]:
test = pd.read_csv('test.csv')
year1 = pd.read_csv('year1.csv')
year2 = pd.read_csv('year2.csv')
year3 = pd.read_csv('year2.csv')
year4 = pd.read_csv('year2.csv')

In [81]:
year1["year"] = 1
year2["year"] = 2
year3["year"] = 3
year4["year"] = 4
test["year"] = 5


データをまとめる

In [82]:
all_data = pd.concat([year1,year2,year3,year4])
all_data.head()

前処理

In [84]:
drop_col = ["TARGET","id"]
for i in all_data.drop(drop_col,axis=1):
    all_data[i] = pd.to_numeric(all_data[i],errors='coerce')#"？"（非数値型）をNaNに置換
all_data = all_data.fillna(all_data.mean())#NaNを中央値に置換

In [85]:
for i in test.drop("id",axis=1):
    test[i] = pd.to_numeric(test[i],errors='coerce')
test = test.fillna(test.mean())

特徴量を増やす

In [116]:
#total_assetsを真数に
all_data["total_assets"] = np.exp(all_data["X29"])
#分母がtotal_assetsの変数にかけて特徴量を増やす
all_data["net_profit"] = all_data["total_assets"] * all_data["X1"]
all_data["total liabilities"] = all_data["total_assets"] * all_data["X2"]
all_data["total liabilities"] = all_data["total_assets"] * all_data["X3"]
all_data["retained earnings"] = all_data["total_assets"] * all_data["X6"]
all_data["gross profit + interest"] = all_data["total_assets"] * all_data["X14"]
all_data["gross profit"] = all_data["total_assets"] * all_data["X18"]
all_data["gross profit (in 3 years)"] = all_data["total_assets"] * all_data["X24"]
all_data["(equity - share capital)"] = all_data["total_assets"] * all_data["X25"]
all_data["profit on sales"] = all_data["total_assets"] * all_data["X35"]
all_data["total sales"] = all_data["total_assets"] * all_data["X36"]
all_data["constant capital"] = all_data["total_assets"] * all_data["X38"]
all_data["EBITDA"] = all_data["total_assets"] * all_data["X48"]
all_data["short-term liabilities"] = all_data["total_assets"] * all_data["X51"]

#testデータも同様に処理
test["total_assets"] = np.exp(test["X29"])

test["net_profit"] = test["total_assets"] * test["X1"]
test["total liabilities"] = test["total_assets"] * test["X2"]
test["total liabilities"] = test["total_assets"] * test["X3"]
test["retained earnings"] = test["total_assets"] * test["X6"]
test["gross profit + interest"] = test["total_assets"] * test["X14"]
test["gross profit"] = test["total_assets"] * test["X18"]
test["gross profit (in 3 years)"] = test["total_assets"] * test["X24"]
test["(equity - share capital)"] = test["total_assets"] * test["X25"]
test["profit on sales"] = test["total_assets"] * test["X35"]
test["total sales"] = test["total_assets"] * test["X36"]
test["constant capital"] = test["total_assets"] * test["X38"]
test["EBITDA"] = test["total_assets"] * test["X48"]
test["short-term liabilities"] = test["total_assets"] * test["X51"]

In [ ]:
取り除く列のリスト
not_use_columns = ['X37']
# 学習用データから列を削除する（PassengerIdは後ほど取り除く）
all_data.drop(not_use_columns, axis=1, inplace=True)
# テスト用データから列を削除する
test.drop(not_use_columns, axis=1, inplace=True)

Xとyの決定

In [94]:
X_1 = all_data.drop(drop_col,axis=1)
y_1 = all_data.loc[:, 'TARGET']

scaler = StandardScaler()
scaler.fit_transform(X_1)

X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, random_state=0)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

pca = PCA(n_components = 20)
X_pca = pca.fit_transform(all_data)

distortions = []

for k  in range(1,20):             
    km = KMeans(n_clusters=k,       
                init='k-means++',
                n_init=10,        
                max_iter=300,    
                random_state=0) 
    km.fit(X_pca)           
    distortions.append(km.inertia_) 

# 結果を散布図に出力

plt.plot(range(1,20), distortions,marker='o') 
plt.xticks([i for i in range(1, 20)])         
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [113]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80, max_features='auto')
rf.fit(X_rf_train, y_rf_train)
print('Training done using Random Forest')
ranking = np.argsort(-rf.feature_importances_)

Training done using Random Forest


In [98]:
cols = X_1.columns.values[ranking][0:40]
X = all_data[cols]
test_X = test[cols]
y = all_data.loc[:, 'TARGET']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.3,random_state=0)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params_xgb = {'max_depth': 4,
              'eta': 0.3,
              'objective': 'binary:logistic',
              'silent':1,
              'random_state':1234, 
              'eval_metric': 'rmse'
             } 
watch_list = [(dtrain, 'train'), (dtest, 'eval')]

model_xgb = xgb.train(params_xgb,
                      dtrain,
                      early_stopping_rounds=20,
                      evals=watch_list)

xgb_predict = model_xgb.predict(xgb.DMatrix(test_X), ntree_limit = model_xgb.best_ntree_limit)

# データを整形
xgb_predict = np.array([[1-i, i] for i in xgb_predict])

In [ ]:
from catboost import CatBoost, CatBoostClassifier
from catboost import Pool

#データをセット
catBoost_train = Pool(X_train, label=y_train) 
catBoost_eval = Pool(X_test, label=y_test)

# パラメータを設定
params_cat = {'loss_function':'Logloss',
              'num_boost_round':1000,
              'early_stopping_rounds':20,
             } 
# 学習 
catb = CatBoost(params_cat)
catb.fit(catBoost_train, eval_set=catBoost_eval ,use_best_model=True, verbose=False)
cat_predict = catb.predict(test_X, prediction_type='Probability') 

cat_predict

In [ ]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, free_raw_data=False)


light_params = {
                'objective': 'binary',
                'metric': 'auc',
                'learning_rate': 0.1, 
                'num_leaves': 40 
               } 

evaluation_results = {}
#学習 
lgb_model = lgb.train(light_params, # 上記で設定したパラメータ
                      lgb_train, # 使用するデータセット 
                      num_boost_round=200, # 学習の回数 
                      valid_names=['train', 'valid'], # 学習経過で表示する名称 
                      valid_sets=[lgb_train, lgb_eval], # モデルの検証に使用するデータセット 
                      evals_result=evaluation_results, # 学習の経過を保存 
                      early_stopping_rounds=20, # アーリーストッピングの回数 
                      verbose_eval=0) # 学習の経過を表示する刻み（非表示）  

# テストデータで予測
lgb_predict = lgb_model.predict(test_X, num_iteration=lgb_model.best_iteration)

In [ ]:
k1 = 1 
k2 = 1 
k3 = 1 

voting_result = [] 

for i in range(len(test)):
    voting = np.argmax(k1*xgb_predict[i] + k2*cat_predict[i] + k3*lgb_predict[i])
    np.round(voting).astype(int) 
    voting_result.append(voting)
    prediction_XG = np.round(voting_result).astype(int)

In [ ]:
pred = prediction_XG

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'TARGET': pred})
submission.to_csv('submission.csv', index=False)

保存

In [ ]:
'''from sklearn.linear_model import LogisticRegression
models = {
    ("RandomForest", RandomForestClassifier()),
    ("GradientBoosting", GradientBoostingClassifier()),
    ("NeuralNetwork", MLPClassifier(max_iter = 10000,)),
    ("AdaBoost", AdaBoostClassifier())
    }
for name, model in models:
    acc = accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))
    print("Accuracy of", name, ":", acc)

from sklearn.ensemble import StackingClassifier
import multiprocessing
cores = multiprocessing.cpu_count()
stacking = StackingClassifier(estimators = models, final_estimator = MLPClassifier(), n_jobs = cores)
acc = accuracy_score(y_test, stacking.fit(X_train, y_train).predict(X_test))
print("Accuracy of Stacking :", acc)
'''

csvに出力